In [ ]:
##################################
# GPUを使用して実行すること！
##################################
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.2 MB/s 


In [17]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from janome.tokenizer import Tokenizer
import os, glob

#Janomeを使って形態素解析
ja_tokenizer=Tokenizer()

#日本語を単語や品詞ごとに分ける
def ja_tokenize(text):
    res=[]
    lines=text.split("\n")
    lines=lines[2:] #最初の2行はヘッダーなので捨てる
    for line in lines:
        malist=ja_tokenizer.tokenize(line)
        for tok in malist:
            ps=tok.part_of_speech.split(",")[0]
            # 他の品詞は無視
            if not ps in ['名詞', '動詞', '形容詞']:continue
            w=tok.base_form
            if w=="*" or w=="": w=tok.surface
            if w=="" or w=="\n": continue
            res.append(w)
        res.append("\n")
    return res

#テストデータを読み込み
root_dir ='/content/drive/MyDrive/text'
for path in glob.glob(root_dir + "/*/*.txt", recursive=True):
    if path.find("LICENSE")>0: continue #LICENSE.txtは除く
    print(path)
    path_wakati=path + ".wakati"
    #ファイルができているときはスルー
    if os.path.exists(path_wakati): continue
    text=open(path,"r", encoding='utf-8').read() #エンコーディングに注意
    words=ja_tokenize(text)
    wt=" ".join(words)
    open(path_wakati, "w", encoding="utf-8").write(wt)

ストリーミング出力は最後の 5000 行に切り捨てられました。
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-5975097.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-5762540.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-5005857.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-6148585.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-4839912.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-6892725.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-6209121.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-6334345.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-6731674.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-5871774.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-6004175.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-6754656.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-6143393.txt
/content/drive/MyDrive/text/dokujo-tsushin/dokujo-tsushin-

In [ ]:
import os, glob, json
 
 
#パスの設定
root_dir ='/content/drive/MyDrive/text'
dic_file=root_dir+"/word-dic.json"
data_file=root_dir+"/textdata.json"
 
#辞書、ハッシュマップ的な
word_dic={"_MAX":0} 
 
 
 
#辞書作成関連の関数たち-------------------------------------------------
 
#辞書に全部の単語を登録する
def register_dic():
    files=glob.glob(root_dir+"/*/*.wakati", recursive=True)
    for i in files:
        file_to_ids(i)
        
#ファイルを読んで固定長シーケンスを返す
def file_to_ids(fname):
    with open(fname, "r", encoding='utf-8') as f:
        text=f.read()
        return text_to_ids(text)
    
#語句を区切ってラベリングする
def text_to_ids(text):
    text=text.strip()
    words=text.split(" ")
    result=[]
    for n in words:
        n=n.strip()
        if n=="": continue
        if not n in word_dic: #まだ登録していない言葉の場合
            wid=word_dic[n]=word_dic["_MAX"]
            word_dic["_MAX"]+=1
            print(wid,n)
        else:
            wid=word_dic[n] #登録済みの言葉の場合
        result.append(wid)
    return result
 
 
 
#ベクトル作成関連の関数たち-------------------------------------------------
 
#ジャンルごとにファイルを読み込む
def count_freq(limit=0):
    X=[]
    Y=[]
    max_words = word_dic["_MAX"]
    cat_names=[]
    for cat in os.listdir(root_dir):
        cat_dir =root_dir + "/" + cat
        if not os.path.isdir(cat_dir):continue #フォルダは無視する
        cat_idx=len(cat_names)
        cat_names.append(cat)
        files=glob.glob(cat_dir+"/*.wakati")
        i=0
        for path in files:
            #print(path)
            cnt=count_file_freq(path)
            X.append(cnt)
            Y.append(cat_idx)
            if limit > 0:
                if i >limit : break
                i+=1
    return X,Y
 
#ファイル内の単語を数える
def count_file_freq(fname):
    cnt=[0 for n in range(word_dic["_MAX"])]
    with open(fname,"r", encoding='utf-8') as f:
        text=f.read().strip()
        ids=text_to_ids(text)
        for wid in ids:
            # ここでエラーになる！
            cnt[wid]+=1
    return cnt
 
#-------------------------------------------
#単語辞書の作成
if os.path.exists(dic_file):
    word_dic =json.load(open(dic_file))
else:
    register_dic()
    json.dump(word_dic, open(dic_file,"w", encoding='utf-8'))
 
#ファイルごとの単語出現頻度のベクトルを作る
print ("要素数=" + str(len(word_dic)))
X, Y=count_freq(100)
json.dump({"X": X, "Y":Y}, open(data_file,"w", encoding='utf-8'))
print("ファイル変換終了")

要素数=1
[]
0
0 楽天
1 岩隈
2 禁断
3 不倫
4 愛
5 ——、
6 相手
7 妻
8 義理
9 妹
10 !?
11 21
12 日
13 発売
14 週刊
15 ポスト
16 今日
17 メジャー
18 移籍
19 向ける
20 FA
21 宣言
22 行う
23 目
24 する
25 れる
26 いる
27 東北
28 ゴールデンイーグルス
29 エース
30 愛妻
31 家
32 知る
33 久志
34 スキャンダル
35 報じる
36 同誌
37 車内
38 キス
39 シーン
40 ゴルフ
41 練習
42 場
43 デート
44 様子
45 合計
46 7
47 ページ
48 渡る
49 掲載
50 同日
51 日本テレビ
52 情報
53 ライブ
54 ミヤネ
55 屋
56 話題
57 触れる
58 テロップ
59 義妹
60 抱擁
61 濃厚
62 紹介
63 親類
64 伝える
65 番組
66 出演
67 芸能
68 レポーター
69 井上
70 公
71 造
72 氏
73 発覚
74 持つ
75 さん
76 形
77 断定
78 できる
79 歯切れ
80 悪い
81 詳細
82 掴める
83 司会
84 宮
85 根
86 誠司
87 家族
88 話し合う
89 頂く
90 いう
91 留める
92 関連
93 リンク
94 星野
95 監督
96 開幕
97 投手
98 決める
99 経緯
100 告白
101 バラ
102 年
103 105
104 億
105 報道
106 一切
107 ない
108 大リーグ
109 ノム
110 困る
111 もん


IndexError: ignored

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# 1：ライブラリのインポート--------------------------------
from sklearn import naive_bayes, metrics, preprocessing #機械学習用のライブラリを利用
import sklearn.model_selection as cross_validation
import json
import numpy  #numpyという行列などを扱うライブラリを利用

In [ ]:
# 2：データ準備
nb_classes=9
data=json.load(open("/content/drive/MyDrive/text/textdata.json"))
X=data["X"] #単語ベクトル
Y=data["Y"] #クラスラベル
max_words=len(X[0]) #最大単語数

In [ ]:
#解説 3：機械学習で分類・識別する---------------------------------------------------
clf=naive_bayes.MultinomialNB(alpha=0.1, fit_prior="True")

NameError: ignored

In [ ]:
# 4：K分割交差検証（cross validation）で性能を評価する---------------------
scores=cross_validation.cross_val_score(clf, X, Y, cv=10)
print("平均正解率 = ", scores.mean())
print("正解率の標準偏差 = ", scores.std())

In [ ]:
# 5：トレーニングデータとテストデータに分ける------------------
#X_train, X_test, Y_train, Y_test=cross_validation.train_test_split(X,Y, test_size=0.5, random_state=0)
#clf.fit(X_train, Y_train)
#print(clf.score(X_test, Y_test)) #テストデータに対する識別結果